In [1]:
import numpy as np
import pandas as pd

from astropy.io import fits
from astropy.table import Table

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

from tqdm.notebook import tqdm
from astropy.visualization import make_lupton_rgb
plt.style.use('dark_background')

In [2]:
def write_fit_file(name, x, data):
    my_types = {'COADD_OBJECT_ID': int, 'TILENAME': str, 'PROB': float, 'HPIX_16384': int, 'HPIX_4096': int,
       'DNF_ZMEAN_SOF': float, 'RA': float, 'DEC': float, 'FLUX_RADIUS_G': float, 'FLUX_RADIUS_R': float,
       'FLUX_RADIUS_I': float, 'FLUX_RADIUS_Z': float, 'KRON_RADIUS': float, 'GAP_FLUX_G': float,
       'MOF_BDF_FLUX_G': float, 'MOF_PSF_FLUX_G': float, 'SOF_BDF_FLUX_G': float, 'SOF_PSF_FLUX_G': float,
       'MAG_AUTO_G': float, 'MAG_APER_4_G': float, 'MAG_APER_8_G': float, 'SOF_BDF_G_1': float,
       'SOF_BDF_G_2': float, 'IMAFLAGS_ISO_G': int, 'IMAFLAGS_ISO_R': int, 'IMAFLAGS_ISO_I': int,
       'IMAFLAGS_ISO_Z': int, 'EXT_COADD': int, 'NEW_ID': int, 'CLASS': str, 'REPORTED': bool,
       'SUBJECT_ID': int, 'SCORE': float, 'PROB_SING': float, 'PROB_DOUB': float}
    
    data = data.astype(my_types)
    
    primary = fits.PrimaryHDU()
    image = fits.ImageHDU(x, name="IMAGE")
    table_data = Table.from_pandas(data)
    table = fits.BinTableHDU(data = table_data)
    hdu_list = fits.HDUList([primary, image, table])
    hdu_list.writeto(filepath + name + '.fits', overwrite=True)   
    hdu_list.close()

In [3]:
filepath = '/Users/jimenagonzalez/research/DSPL/SpaceWarps_Inspection/Results/fits_files/'

hdu_list = fits.open(filepath + 'all_subjects.fits')
images = hdu_list[1].data
data = pd.DataFrame(hdu_list[2].data)
hdu_list.close()

data_tmp = data[data['SCORE'] > 0.00001]
images_tmp = images[data_tmp.index]
print(len(data_tmp))

write_fit_file('candidates_experts', images_tmp, data_tmp)

2502


In [4]:
def make_one_png(objects, title, data):
    Q_list, stre_list = [9, 8, 10.5], [42, 35, 37]
    min_list = [(0, 0, 0), (14, 2, 0), (14, 8, 0)]
    
    plt.figure(figsize=(5.4,5.4))
    for i in range(3):
        plt.subplot(2,2,i+1)        
        #plt.title(title)
        #print(data['COADD_OBJECT_ID'].iloc[i+j]) 
        rgb = make_lupton_rgb(objects[2], objects[1], objects[0], 
                            Q=Q_list[i], stretch=stre_list[i], minimum=min_list[i])
        rgb = np.flipud(rgb)
        plt.imshow(rgb, aspect='equal')
        plt.xticks([], [])
        plt.yticks([], []) 
        
    #bluest    
    plt.subplot(2,2,4)        
    imager = 0.0005*objects[2] 
    imageg = 3*objects[1] 
    imageb = 15*objects[0] 
    rgb = np.array([imager, imageg, imageb])
    #Making sure values are lower or equal to 255
    rgb = np.clip(rgb, 0, 255).astype(np.uint8)
    rgb = rgb.transpose(1, 2, 0)
    #Increasing contrast
    min_img, max_img = np.min(rgb), np.max(rgb)     
    LUT=np.zeros(256,dtype=np.uint8)
    LUT[min_img:max_img+1]=np.linspace(start=0,stop=255,num=(max_img-min_img)+1,endpoint=True,dtype=np.uint8)
    rgb = LUT[rgb]
    rgb = np.flipud(rgb)
    
    plt.imshow(rgb, aspect='equal')
    plt.xticks([], [])
    plt.yticks([], []) 
    
    plt.savefig(png_filepath + title + '.png', bbox_inches='tight')
    #plt.show() 
    plt.close()

In [5]:
# Candidates above threshold
hdu_list = fits.open(filepath + 'candidates_experts.fits')
images_cand = hdu_list[1].data
data_cand = pd.DataFrame(hdu_list[2].data)
hdu_list.close()
print(len(data_cand))

# Additional candidates
hdu_list = fits.open(filepath + 'additional.fits')
images_addi = hdu_list[1].data
data_addi = pd.DataFrame(hdu_list[2].data)
hdu_list.close()
print(len(data_addi))

# Random candidates below the threshold
hdu_list = fits.open(filepath + 'random.fits')
images_rand = hdu_list[1].data
data_rand = pd.DataFrame(hdu_list[2].data)
hdu_list.close()
print(len(data_rand))

images = np.concatenate((images_cand, images_addi, images_rand)).astype(np.float32)
data = pd.concat([data_cand, data_addi, data_rand], axis=0).reset_index(drop=True)
data = data.drop_duplicates() #removing duplicates
data = data.sample(frac=1, random_state=42) #random order

images = images[data.index]
data = data.reset_index(drop=True)

print(len(images), len(data))

2502
116
500
3106 3106


In [6]:
png_filepath = '/Users/jimenagonzalez/research/DSPL/SpaceWarps_Inspection/Results/For_experts/images/'
file_path = "File_list"

with open(file_path, 'w') as file:
    for i in tqdm(range(len(data))):
        title = str(data['SUBJECT_ID'].iloc[i]) 
        make_one_png(images[i], title, data.iloc[i])
        file.write(f"{title}.png\n")

  0%|          | 0/3106 [00:00<?, ?it/s]